In [ ]:
# Import altair first
import altair as alt
from onecodex.notebooks.report import title

# Necessary to trigger our renderer/theme
from onecodex import Api
ocx = Api()

In [ ]:
title("Example Report")

In [ ]:
# import altair as alt
from vega_datasets import data

source = data.stocks()

alt.Chart(source).mark_area(
    color="lightblue",
    interpolate='step-after',
    line=True
).encode(
    x='date',
    y='price'
).transform_filter(alt.datum.symbol == 'GOOG')
